In [1]:
import pandas as pd
import numpy as np
import json, csv, datetime
import requests
import os
from pathlib import Path
import pickle
from shutil import unpack_archive

In [2]:
#input

nab_path = '/home/adrian/Dokumente/real_data/rnn/'
 
urls = dict()
urls['ecg']=['http://www.cs.ucr.edu/~eamonn/discords/ECG_data.zip',
             'http://www.cs.ucr.edu/~eamonn/discords/mitdbx_mitdbx_108.txt',
             'http://www.cs.ucr.edu/~eamonn/discords/qtdbsele0606.txt',
             'http://www.cs.ucr.edu/~eamonn/discords/chfdbchf15.txt',
             'http://www.cs.ucr.edu/~eamonn/discords/qtdbsel102.txt']
urls['gesture']=['http://www.cs.ucr.edu/~eamonn/discords/ann_gun_CentroidA']
urls['space_shuttle']=['http://www.cs.ucr.edu/~eamonn/discords/TEK16.txt',
                       'http://www.cs.ucr.edu/~eamonn/discords/TEK17.txt',
                       'http://www.cs.ucr.edu/~eamonn/discords/TEK14.txt']
urls['respiration']=['http://www.cs.ucr.edu/~eamonn/discords/nprs44.txt',
                     'http://www.cs.ucr.edu/~eamonn/discords/nprs43.txt']
urls['power_demand']=['http://www.cs.ucr.edu/~eamonn/discords/power_data.txt']


#output
output_path = '/home/adrian/Dokumente/real_data/rnn'


In [3]:
for dataname in urls:
    raw_dir = Path(output_path, dataname, 'raw')
    raw_dir.mkdir(parents=True, exist_ok=True)
    for url in urls[dataname]:
        filename = raw_dir.joinpath(Path(url).name)
        print('Downloading', url)
        resp =requests.get(url)
        filename.write_bytes(resp.content)
        if filename.suffix=='':
            filename.rename(filename.with_suffix('.txt'))
        print('Saving to', filename.with_suffix('.txt'))
        if filename.suffix=='.zip':
            print('Extracting to', filename)
            unpack_archive(str(filename), extract_dir=str(raw_dir))

Saving to /home/adrian/Dokumente/real_data/rnn/ecg/raw/ECG_data.txt
Extracting to /home/adrian/Dokumente/real_data/rnn/ecg/raw/ECG_data.zip
Saving to /home/adrian/Dokumente/real_data/rnn/ecg/raw/mitdbx_mitdbx_108.txt
Saving to /home/adrian/Dokumente/real_data/rnn/ecg/raw/qtdbsele0606.txt
Saving to /home/adrian/Dokumente/real_data/rnn/ecg/raw/chfdbchf15.txt
Saving to /home/adrian/Dokumente/real_data/rnn/ecg/raw/qtdbsel102.txt
Saving to /home/adrian/Dokumente/real_data/rnn/gesture/raw/ann_gun_CentroidA.txt
Saving to /home/adrian/Dokumente/real_data/rnn/space_shuttle/raw/TEK16.txt
Saving to /home/adrian/Dokumente/real_data/rnn/space_shuttle/raw/TEK17.txt
Saving to /home/adrian/Dokumente/real_data/rnn/space_shuttle/raw/TEK14.txt
Saving to /home/adrian/Dokumente/real_data/rnn/respiration/raw/nprs44.txt
Saving to /home/adrian/Dokumente/real_data/rnn/respiration/raw/nprs43.txt
Saving to /home/adrian/Dokumente/real_data/rnn/power_demand/raw/power_data.txt


In [10]:
for dataname in urls:
    raw_dir = Path(output_path, dataname, 'raw')
    for filepath in raw_dir.glob('*.txt'):
        print(filepath)
        with open(str(filepath)) as f:
            # Label anomaly points as 1 in the dataset
            labeled_data=[]
            for i, line in enumerate(f):
                tokens = [float(token) for token in line.split()]
                if raw_dir.parent.name== 'ecg':
                    # Remove time-step channel
                    tokens.pop(0)
                if filepath.name == 'chfdbchf15.txt':
                    tokens.append(1.0) if 2250 < i < 2400 else tokens.append(0.0)
                elif filepath.name == 'xmitdb_x108_0.txt':
                    tokens.append(1.0) if 4020 < i < 4400 else tokens.append(0.0)
                elif filepath.name == 'mitdb__100_180.txt':
                    tokens.append(1.0) if 1800 < i < 1990 else tokens.append(0.0)
                elif filepath.name == 'chfdb_chf01_275.txt':
                    tokens.append(1.0) if 2330 < i < 2600 else tokens.append(0.0)
                elif filepath.name == 'ltstdb_20221_43.txt':
                    tokens.append(1.0) if 650 < i < 780 else tokens.append(0.0)
                elif filepath.name == 'ltstdb_20321_240.txt':
                    tokens.append(1.0) if 710 < i < 850 else tokens.append(0.0)
                elif filepath.name == 'chfdb_chf13_45590.txt':
                    tokens.append(1.0) if 2800 < i < 2960 else tokens.append(0.0)
                elif filepath.name == 'stdb_308_0.txt':
                    tokens.append(1.0) if 2290 < i < 2550 else tokens.append(0.0)
                elif filepath.name == 'qtdbsel102.txt':
                    tokens.append(1.0) if 4230 < i < 4430 else tokens.append(0.0)
                elif filepath.name == 'ann_gun_CentroidA.txt':
                    tokens.append(1.0) if 2070 < i < 2810 else tokens.append(0.0)
                elif filepath.name == 'TEK16.txt':
                    tokens.append(1.0) if 4270 < i < 4370 else tokens.append(0.0)
                elif filepath.name == 'TEK17.txt':
                    tokens.append(1.0) if 2100 < i < 2145 else tokens.append(0.0)
                elif filepath.name == 'TEK14.txt':
                    tokens.append(1.0) if 1100 < i < 1200 or 1455 < i < 1955 else tokens.append(0.0)
                elif filepath.name == 'nprs44.txt':
                    tokens.append(1.0) if 16192 < i < 16638 or 20457 < i < 20911 else tokens.append(0.0)
                elif filepath.name == 'nprs43.txt':
                    tokens.append(1.0) if 12929 < i < 13432 or 14877 < i < 15086 or 15729 < i < 15924 else tokens.append(0.0)
                elif filepath.name == 'power_data.txt':
                    tokens.append(1.0) if 8254 < i < 8998 or 11348 < i < 12143 or 33883 < i < 34601 else tokens.append(0.0)
                labeled_data.append(tokens)

            # Fill in the point where there is no signal value.
            if filepath.name == 'ann_gun_CentroidA.txt':
                for i, datapoint in enumerate(labeled_data):
                    for j,channel in enumerate(datapoint[:-1]):
                        if channel == 0.0:
                            labeled_data[i][j] = 0.5 * labeled_data[i - 1][j] + 0.5 * labeled_data[i + 1][j]

            # Save the labeled dataset as .pkl extension
            labeled_whole_dir = raw_dir.parent.joinpath('labeled', 'whole')
            labeled_whole_dir.mkdir(parents=True, exist_ok=True)
            with open(str(labeled_whole_dir.joinpath(filepath.name).with_suffix('.csv')), 'wb') as pkl:
                #pickle.dump(labeled_data, pkl)
                print(labeled_data)
            
            """
            # Divide the labeled dataset into trainset and testset, then save them
            labeled_train_dir = raw_dir.parent.joinpath('labeled','train')
            labeled_train_dir.mkdir(parents=True,exist_ok=True)
            labeled_test_dir = raw_dir.parent.joinpath('labeled','test')
            labeled_test_dir.mkdir(parents=True,exist_ok=True)
            if filepath.name == 'chfdb_chf13_45590.txt':
                with open(str(labeled_train_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    print(labeled_data[:2439])
                    pickle.dump(labeled_data[:2439], pkl)
                with open(str(labeled_test_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[2439:3726], pkl)
            elif filepath.name == 'chfdb_chf01_275.txt':
                with open(str(labeled_train_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[:1833], pkl)
                with open(str(labeled_test_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[1833:3674], pkl)
            elif filepath.name == 'chfdbchf15.txt':
                with open(str(labeled_train_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[3381:14244], pkl)
                with open(str(labeled_test_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[33:3381], pkl)
            elif filepath.name == 'qtdbsel102.txt':
                with open(str(labeled_train_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[10093:44828], pkl)
                with open(str(labeled_test_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[211:10093], pkl)
            elif filepath.name == 'mitdb__100_180.txt':
                with open(str(labeled_train_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[2328:5271], pkl)
                with open(str(labeled_test_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[73:2328], pkl)
            elif filepath.name == 'stdb_308_0.txt':
                with open(str(labeled_train_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[2986:5359], pkl)
                with open(str(labeled_test_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[265:2986], pkl)
            elif filepath.name == 'ltstdb_20321_240.txt':
                with open(str(labeled_train_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[1520:3531], pkl)
                with open(str(labeled_test_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[73:1520], pkl)
            elif filepath.name == 'xmitdb_x108_0.txt':
                with open(str(labeled_train_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[424:3576], pkl)
                with open(str(labeled_test_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[3576:5332], pkl)
            elif filepath.name == 'ltstdb_20221_43.txt':
                with open(str(labeled_train_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[1121:3731], pkl)
                with open(str(labeled_test_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[0:1121], pkl)
            elif filepath.name == 'ann_gun_CentroidA.txt':
                with open(str(labeled_train_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[3000:], pkl)
                with open(str(labeled_test_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[:3000], pkl)
            elif filepath.name == 'nprs44.txt':
                with open(str(labeled_train_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[363:12955], pkl)
                with open(str(labeled_test_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[12955:24082], pkl)
            elif filepath.name == 'nprs43.txt':
                with open(str(labeled_train_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[4285:10498], pkl)
                with open(str(labeled_test_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[10498:17909], pkl)
            elif filepath.name == 'power_data.txt':
                with open(str(labeled_train_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[15287:33432], pkl)
                with open(str(labeled_test_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[501:15287], pkl)
            elif filepath.name == 'TEK17.txt':
                with open(str(labeled_train_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[2469:4588], pkl)
                with open(str(labeled_test_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[1543:2469], pkl)
            elif filepath.name == 'TEK16.txt':
                with open(str(labeled_train_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[521:3588], pkl)
                with open(str(labeled_test_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[3588:4539], pkl)
            elif filepath.name == 'TEK14.txt':
                with open(str(labeled_train_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[2089:4098], pkl)
                with open(str(labeled_test_dir.joinpath(filepath.name).with_suffix('.pkl')), 'wb') as pkl:
                    pickle.dump(labeled_data[97:2089], pkl)"""

/home/adrian/Dokumente/real_data/rnn/ecg/raw/chfdbchf15.txt
[[-0.215, 0.015, 0.0], [-0.24, 0.03, 0.0], [-0.24, 0.005, 0.0], [-0.225, -0.02, 0.0], [-0.225, -0.05, 0.0], [-0.2, -0.055, 0.0], [-0.23, -0.05, 0.0], [-0.28, -0.05, 0.0], [-0.365, 0.0, 0.0], [-0.41, -0.025, 0.0], [-0.455, -0.025, 0.0], [-0.5, -0.005, 0.0], [-0.55, 0.015, 0.0], [-0.555, -0.01, 0.0], [-0.545, -0.04, 0.0], [-0.55, -0.02, 0.0], [-0.575, -0.025, 0.0], [-0.585, -0.025, 0.0], [-0.58, -0.025, 0.0], [-0.56, -0.035, 0.0], [-0.58, -0.045, 0.0], [-0.62, -0.025, 0.0], [-0.635, -0.015, 0.0], [-0.57, -0.035, 0.0], [-0.405, -0.065, 0.0], [-0.17, -0.07, 0.0], [0.13, -0.005, 0.0], [0.39, 0.08, 0.0], [0.695, 0.01, 0.0], [1.08, -0.35, 0.0], [1.615, -0.985, 0.0], [2.15, -1.625, 0.0], [2.505, -2.265, 0.0], [2.51, -2.845, 0.0], [2.345, -3.21, 0.0], [2.08, -3.125, 0.0], [1.67, -2.55, 0.0], [1.18, -1.915, 0.0], [0.695, -1.405, 0.0], [0.28, -1.055, 0.0], [-0.09, -0.785, 0.0], [-0.42, -0.575, 0.0], [-0.695, -0.445, 0.0], [-0.855, -0.35,

[[4.77, 2.1, 0.0], [4.82, 2.135, 0.0], [4.805, 2.19, 0.0], [4.75, 2.285, 0.0], [4.71, 2.44, 0.0], [4.695, 2.62, 0.0], [4.685, 2.805, 0.0], [4.675, 3.045, 0.0], [4.68, 3.365, 0.0], [4.685, 3.695, 0.0], [4.675, 3.97, 0.0], [4.67, 4.24, 0.0], [4.67, 4.53, 0.0], [4.68, 4.8, 0.0], [4.665, 5.025, 0.0], [4.645, 5.215, 0.0], [4.635, 5.4, 0.0], [4.635, 5.575, 0.0], [4.655, 5.715, 0.0], [4.66, 5.795, 0.0], [4.65, 5.865, 0.0], [4.66, 5.925, 0.0], [4.675, 5.98, 0.0], [4.675, 6.02, 0.0], [4.655, 6.06, 0.0], [4.66, 6.08, 0.0], [4.68, 6.145, 0.0], [4.675, 6.18, 0.0], [4.655, 6.22, 0.0], [4.65, 6.275, 0.0], [4.675, 6.32, 0.0], [4.665, 6.355, 0.0], [4.645, 6.38, 0.0], [4.64, 6.45, 0.0], [4.655, 6.49, 0.0], [4.645, 6.53, 0.0], [4.625, 6.575, 0.0], [4.625, 6.645, 0.0], [4.635, 6.7, 0.0], [4.65, 6.735, 0.0], [4.625, 6.78, 0.0], [4.635, 6.835, 0.0], [4.65, 6.885, 0.0], [4.66, 6.92, 0.0], [4.65, 6.945, 0.0], [4.65, 6.985, 0.0], [4.66, 7.01, 0.0], [4.68, 7.035, 0.0], [4.675, 7.05, 0.0], [4.695, 7.065, 0.0], 

[[4.995, 4.34], [4.995, 4.34], [4.995, 4.34], [4.995, 4.34], [4.995, 4.34], [4.995, 4.34], [4.995, 4.34], [4.995, 4.32], [4.985, 4.31], [4.995, 4.28], [4.995, 4.29], [5.0, 4.3], [5.025, 4.305], [5.035, 4.31], [5.015, 4.32], [4.95, 4.305], [4.895, 4.285], [4.865, 4.28], [4.84, 4.3], [4.825, 4.31], [4.78, 4.295], [4.725, 4.27], [4.71, 4.26], [4.715, 4.275], [4.74, 4.28], [4.75, 4.275], [4.73, 4.27], [4.71, 4.255], [4.695, 4.245], [4.7, 4.265], [4.715, 4.27], [4.75, 4.265], [4.745, 4.265], [4.73, 4.27], [4.71, 4.255], [4.715, 4.255], [4.725, 4.265], [4.745, 4.27], [4.73, 4.275], [4.725, 4.275], [4.725, 4.26], [4.725, 4.27], [4.735, 4.28], [4.74, 4.29], [4.735, 4.28], [4.72, 4.265], [4.715, 4.26], [4.715, 4.265], [4.715, 4.265], [4.705, 4.265], [4.685, 4.265], [4.695, 4.25], [4.695, 4.25], [4.73, 4.265], [4.745, 4.275], [4.755, 4.29], [4.75, 4.305], [4.755, 4.295], [4.74, 4.285], [4.715, 4.265], [4.715, 4.215], [4.73, 4.17], [4.755, 4.075], [4.79, 3.98], [4.8, 3.86], [4.825, 3.685], [4.85,

[[-0.22, 0.0], [0.02, 0.0], [-0.22, 0.0], [0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [0.02, 0.0], [-0.22, 0.0], [0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [0.02, 0.0], [-0.22, 0.0], [0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [0.02, 0.0], [-0.22, 0.0], [0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22, 0.0], [-0.02, 0.0], [-0.22

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
nyc_taxi_raw_path = Path(output_path+'/nyc_taxi/raw/nyc_taxi.csv')
labeled_data = []
with open(str(nyc_taxi_raw_path),'r') as f:
    for i, line in enumerate(f):
        tokens = [float(token) for token in line.strip().split(',')[1:]]
        tokens.append(1) if 150 < i < 250 or   \
                            5970 < i < 6050 or \
                            8500 < i < 8650 or \
                            8750 < i < 8890 or \
                            10000 < i < 10200 or \
                            14700 < i < 14800 \
                          else tokens.append(0)
        labeled_data.append(tokens)
nyc_taxi_train_path = nyc_taxi_raw_path.parent.parent.joinpath('labeled','train',nyc_taxi_raw_path.name).with_suffix('.pkl')
nyc_taxi_train_path.parent.mkdir(parents=True, exist_ok=True)
with open(str(nyc_taxi_train_path),'wb') as pkl:
    pickle.dump(labeled_data[:13104], pkl)

nyc_taxi_test_path = nyc_taxi_raw_path.parent.parent.joinpath('labeled','test',nyc_taxi_raw_path.name).with_suffix('.pkl')
nyc_taxi_test_path.parent.mkdir(parents=True, exist_ok=True)
with open(str(nyc_taxi_test_path),'wb') as pkl:
    pickle.dump(labeled_data[13104:], pkl)

In [61]:
labels_file = open(nab_path + labels_filename, 'r')
labels = json.loads(labels_file.read())
labels_file.close()

def load_data_frame_with_labels(ts_name, file_name):
    df = pd.read_csv(nab_data_path + file_name)
    df.insert(0, 'ts_name', ts_name)
    df.insert(2, 'unit', "Value")
    df['class'] = df['timestamp'].isin(labels[file_name]).astype(int)
    df = df.rename(columns={'timestamp': 'time'})
    return df
 
#training_data_frame = load_data_frame_with_labels(training_file_name)
#test_data_frame = load_data_frame_with_labels("Test", file_name_UPS)

In [69]:
def init_file(outputfile):
    with open(outputfile, 'w') as file_output:
        writer = csv.writer(file_output, delimiter=';')
        header = ['ts_name', 'time', 'unit', 'value', 'class']
        writer.writerow(header)
        
def append_to_file(df, outputfile):
    with open(outputfile, 'a') as file_output:
        writer = csv.writer(file_output, delimiter=';')
        for index, row in df.iterrows():
            row = [row[0], datetime.datetime.strptime(row[1], '%Y-%m-%d %H:%M:%S').isoformat(), row[2], row[3], row[4]]
            writer.writerow(row)

In [76]:
init_file(test_file_name)
init_file(train_file_name)

In [77]:
for file in files:
    df = load_data_frame_with_labels(file['name'], file['file_name'])
    df_test = df.iloc[0:10000]
    df_train = df.iloc[10001:15801]
    append_to_file(df_test, test_file_name)
    append_to_file(df_train, train_file_name)

In [54]:
df.iloc[10001:15801]

,ts_name,time,unit,value,class
10001,UPS,2015-04-02 15:07:53,Value,2,0
10002,UPS,2015-04-02 15:12:53,Value,1,0
10003,UPS,2015-04-02 15:17:53,Value,5,0
10004,UPS,2015-04-02 15:22:53,Value,1,0
10005,UPS,2015-04-02 15:27:53,Value,2,0
...,...,...,...,...,...
15796,UPS,2015-04-22 18:02:53,Value,1,0
15797,UPS,2015-04-22 18:07:53,Value,2,0
15798,UPS,2015-04-22 18:12:53,Value,1,0
15799,UPS,2015-04-22 18:17:53,Value,5,0


In [47]:
test_data_frame

,ts_name,time,unit,value,class
0,Test,2015-02-26 21:42:53,Value,2,0
1,Test,2015-02-26 21:47:53,Value,2,0
2,Test,2015-02-26 21:52:53,Value,4,0
3,Test,2015-02-26 21:57:53,Value,3,0
4,Test,2015-02-26 22:02:53,Value,5,0
...,...,...,...,...,...
15861,Test,2015-04-22 23:27:53,Value,3,0
15862,Test,2015-04-22 23:32:53,Value,2,0
15863,Test,2015-04-22 23:37:53,Value,2,0
15864,Test,2015-04-22 23:42:53,Value,3,0


In [23]:
test_data_frame.loc[test_data_frame['class']==1]

,ts_name,time,unit,value,class
1993,TEST,2015-03-05 19:47:53,Value,716,1
3735,TEST,2015-03-11 20:57:53,Value,1673,1
9795,TEST,2015-04-01 21:57:53,Value,282,1
11606,TEST,2015-04-08 04:52:53,Value,211,1
